In [1]:
from collections import defaultdict
from matplotlib.markers import MarkerStyle
from pathlib import Path
from nuclei.coco_exporter import *
from nuclei.imports import *
from nuclei.labelbox_utils import *
import pandas as pd
import json

root_dir = Path('../../').absolute()

from IPython.core.display import display, HTML

Successfully loaded API key: eyJhbGciOi...


In [2]:
project_names = ['IVT_CS', 'IVT_LB', 'IVT_LG', 'IVT_JS', 'IVT_KF']

## Split annotations by class

### Step 1: Read file from all assessors

In [12]:
df_all = pd.read_json(Path(f'../../data/IVT/IVT_all.json'))
df = df_all.T.copy(deep=True)
df = df.drop(columns=['url', 'dataset'])
df.reset_index(drop=True, inplace=True)

In [13]:
len(df)

200

In [14]:
df.head(1)

,file_name,IVT_CS,IVT_LB,IVT_LG,IVT_JS,IVT_KF
0,T19_Da16.jpg,"{'Macrophage': [{'geometry': {'x': 255, 'y': 894}}, {'geometry': {'x': 741, 'y': 55}}, {'geometry': {'x': 132, 'y': 203}}, {'geometry': {'x': 258, 'y': 385}}, {'geometry': {'x': 71, 'y': 54}}, {'geometry': {'x': 862, 'y': 561}}, {'geometry': {'x': 142, 'y': 733}}, {'geometry': {'x': 768, 'y': 353}}, {'geometry': {'x': 916, 'y': 263}}], 'Monocyte': [{'geometry': {'x': 387, 'y': 198}}, {'geometry': {'x': 226, 'y': 600}}], 'Eosinophil': [{'geometry': {'x': 555, 'y': 388}}, {'geometry': {'x': 594, 'y': 451}}, {'geometry': {'x': 163, 'y': 898}}], 'Lymphocyte': [{'geometry': {'x': 836, 'y': 474}...","{'Macrophage': [{'geometry': {'x': 239, 'y': 891}}, {'geometry': {'x': 860, 'y': 144}}], 'Monocyte': [{'geometry': {'x': 71, 'y': 46}}, {'geometry': {'x': 718, 'y': 948}}, {'geometry': {'x': 409, 'y': 695}}, {'geometry': {'x': 958, 'y': 900}}, {'geometry': {'x': 166, 'y': 736}}, {'geometry': {'x': 764, 'y': 358}}, {'geometry': {'x': 139, 'y': 203}}, {'geometry': {'x': 369, 'y': 403}}, {'geometry': {'x': 374, 'y': 205}}, {'geometry': {'x': 238, 'y': 599}}], 'Neutrophil': [{'geometry': {'x': 536, 'y': 379}}, {'geometry': {'x': 170, 'y': 898}}, {'geometry': {'x': 858, 'y': 742}}], 'Lymphocyte...","{'Macrophage': [{'geometry': {'x': 254, 'y': 902}}, {'geometry': {'x': 134, 'y': 196}}, {'geometry': {'x': 390, 'y': 190}}, {'geometry': {'x': 139, 'y': 729}}, {'geometry': {'x': 970, 'y': 884}}, {'geometry': {'x': 839, 'y': 474}}], 'Monocyte': [{'geometry': {'x': 226, 'y': 599}}, {'geometry': {'x': 725, 'y': 940}}, {'geometry': {'x': 385, 'y': 386}}], 'Neutrophil': [{'geometry': {'x': 550, 'y': 387}}, {'geometry': {'x': 163, 'y': 899}}], 'Eosinophil': [{'geometry': {'x': 592, 'y': 449}}], 'Lymphocyte': [{'geometry': {'x': 689, 'y': 389}}], 'Unknown cell': [{'geometry': {'x': 916, 'y': 261...","{'Macrophage': [{'geometry': {'x': 137, 'y': 737}}, {'geometry': {'x': 217, 'y': 593}}, {'geometry': {'x': 840, 'y': 165}}, {'geometry': {'x': 777, 'y': 346}}, {'geometry': {'x': 749, 'y': 51}}, {'geometry': {'x': 131, 'y': 201}}, {'geometry': {'x': 80, 'y': 63}}, {'geometry': {'x': 384, 'y': 389}}, {'geometry': {'x': 426, 'y': 700}}, {'geometry': {'x': 971, 'y': 886}}, {'geometry': {'x': 378, 'y': 183}}, {'geometry': {'x': 843, 'y': 463}}, {'geometry': {'x': 729, 'y': 963}}, {'geometry': {'x': 913, 'y': 451}}], 'Monocyte': [{'geometry': {'x': 768, 'y': 811}}], 'Neutrophil': [{'geometry': ...",Skip


### Optional: Filter by Assessor  (ie 'IVT_CS")

In [ ]:
df.dropna(subset=['IVT_CS'], inplace=True)

In [ ]:
len(df)

### Step 2: select a class

lbl_key = ['Macrophage', 'Monocyte', 'Neutrophil', 'Eosinophil', 'Lymphocyte', 'Epithelial cell', 'Unknown cell']

In [6]:
cl = 'Macrophage'

In [12]:
cl = 'Monocyte'

In [18]:
cl = 'Neutrophil'

In [24]:
cl = 'Eosinophil'

In [30]:
cl = 'Lymphocyte'

In [6]:
cl = 'Epithelial cell'

In [15]:
cl = 'Unknown cell'

### Step 3: save a class in .csv format

In [16]:
assessors = project_names

cl_dict = {}
for k, v in df.iterrows():
    filename = df['file_name'][k]
    tmp = []
    for assessor in assessors:
        try: 
            tmp.append(len(df[assessor][k][cl]))
        except:
            tmp.append(0)
    cl_dict[filename] = tmp

df = pd.DataFrame(cl_dict, index = assessors)

df = df.T
df.sort_index(inplace=True)
df.columns.name = cl
df.head(2)

df.to_csv(Path(f'../../data/IVT/IVT_{cl}.csv'), index_label=cl)

In [17]:
df.head(10)

Unknown cell,IVT_CS,IVT_LB,IVT_LG,IVT_JS,IVT_KF
T01_Da0.jpg,0,0,1,0,0
T01_Da1.jpg,0,0,0,0,0
T01_Da10.jpg,0,2,0,0,0
T01_Da11.jpg,0,2,2,0,0
T01_Da12.jpg,0,0,1,0,0
T01_Da13.jpg,0,0,1,0,0
T01_Da14.jpg,0,0,0,0,0
T01_Da15.jpg,0,0,0,0,0
T01_Da16.jpg,0,0,1,0,0
T01_Da17.jpg,0,0,0,0,0


### Optional: Step 4: Retrieve a class from .csv into a DataFrame

In [18]:
df_cl = pd.read_csv(Path(f'../../data/IVT/IVT_{cl}.csv'), index_col=cl)
df_cl.head()

,IVT_CS,IVT_LB,IVT_LG,IVT_JS,IVT_KF
Unknown cell,,,,,
T01_Da0.jpg,0,0,1,0,0
T01_Da1.jpg,0,0,0,0,0
T01_Da10.jpg,0,2,0,0,0
T01_Da11.jpg,0,2,2,0,0
T01_Da12.jpg,0,0,1,0,0


In [19]:
len(df_cl)

200